In [43]:
import numpy as np

#actions:
# 0 = up, 1 = right, 2 = down, 3 = left
actions = [0, 1, 2, 3]

row = 11 #Dynamic later, this is for testing
col = 11 #Dynamic later, this is for testing
wall = -100
cheese = 100
floor = -1

#rewards and environment mapping:
rewards = np.full((row, col), wall)
rewards[0,5] = cheese  #cheese location for testing, change later
envMap = {}  #mapping the maze into a dictionary, will be changed later to fit unity project
envMap[1] = [i for i in range(1, 10)]
envMap[2] = [1, 7, 9]
envMap[3] = [i for i in range(1, 8)]
envMap[3].append(9)
envMap[4] = [3, 7]
envMap[5] = [i for i in range (11)]
envMap[6] = [5]
envMap[7] = [i for i in range(1, 10)]
envMap[8] = [3, 7]
envMap[9] = [i for i in range(11)]
#set rewards for aisle locations:
for row_index in range(1, 10):
    for column_index in envMap[row_index]:
        rewards[row_index, column_index] = -1

for row in rewards:
    print(row)


[-100 -100 -100 -100 -100  100 -100 -100 -100 -100 -100]
[-100   -1   -1   -1   -1   -1   -1   -1   -1   -1 -100]
[-100   -1 -100 -100 -100 -100 -100   -1 -100   -1 -100]
[-100   -1   -1   -1   -1   -1   -1   -1 -100   -1 -100]
[-100 -100 -100   -1 -100 -100 -100   -1 -100 -100 -100]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-100 -100 -100 -100 -100   -1 -100 -100 -100 -100 -100]
[-100   -1   -1   -1   -1   -1   -1   -1   -1   -1 -100]
[-100 -100 -100   -1 -100 -100 -100   -1 -100 -100 -100]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100]


In [44]:
#define helper functions:
#this delcares that if the rewards for the location is -1, then it is not a terminal state:
def is_terminal_state(current_row_index, current_column_index):
    if rewards[current_row_index, current_column_index] == -1:
        return False
    else:
        return True
#this function chooses a random, non-terminal starting location
def get_starting_location():
    #get a random row and column index:
    current_row_index = np.random.randint(11)
    current_column_index = np.random.randint(11)
    #choose until non-terminal state is found
    while is_terminal_state(current_row_index, current_column_index):
        current_row_index = np.random.randint(0, high=11)
        current_column_index = np.random.randint(0, high=11)
    return current_row_index, current_column_index
#define an epsilon greedy algorithm that will choose which action to take next
def get_next_action(current_row_index, current_column_index, epsilon):
    #if a randomly chosen value between 0 and 1 is less than epsilon, choose the most promising value from qtable
    if np.random.random() < epsilon:
        q_values[0,0]
        return np.argmax(q_values[current_row_index, current_column_index])
    else:
        return np.random.randint(4)
#define a function that will get the next location beased on the chosent action
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    if actions[action_index] == 0 and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 1 and current_column_index < col - 1:
        new_column_index += 1
    elif actions[action_index] == 2 and current_row_index < row -1:
        new_row_index += 1
    elif actions[action_index] == 3 and current_column_index > 0:
        new_column_index -= 1
    return new_row_index, new_column_index
#define a function that will get the shortest path
def get_shortest_path(start_row_index, start_column_index):
    #return immediately if this is an invalid starting location
    if is_terminal_state(start_row_index, start_column_index):
        return []
    else:
        current_row_index, current_column_index = start_row_index, start_column_index
        shortest_path = []
        shortest_path.append([current_row_index, current_column_index])
        #continue moving along path until goal is reached
        while not is_terminal_state(current_row_index, current_column_index):
            action_index = get_next_action(current_row_index, current_column_index, 1)
            current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
            shortest_path.append([current_row_index, current_column_index])
    return shortest_path

In [45]:
#defining the training parameters:
epsilon = 0.9 #this defines a 90% rate of choosing the best action, and a 10% rate of choosing a random action
discount_factor = 0.9 #the discount rate for future rewards
learning_rate = 0.9 #the rate that the agent learns
#set traning epochs and train the agent:
for epoch in range(10):
    row_index, column_index = get_starting_location()
    print(row_index)
    print(column_index)
    while not is_terminal_state(row_index, column_index):
        action_index = get_next_action(row_index, column_index, epsilon)
        old_row_index, old_column_index = row_index, column_index #stores old row and column information
        row_index, column_index = get_next_location(row_index, column_index, action_index)
        reward = rewards[row_index, column_index]
        old_q_value = q_values[old_row_index, old_column_index, action_index]
        temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value
        new_q_value = old_q_value + (learning_rate * temporal_difference)
        q_values[old_row_index, old_column_index, action_index] = new_q_value
print('Training complete!')

7
5


TypeError: list indices must be integers or slices, not tuple